# Exploratory notebook for IMACLIM data-use within MATMat-Trade to input in MatMat

### Imports 

In [3]:
# make the necessary imports
import tqdm
import shutil
import warnings
import pandas as pd
import numpy as np
import multiprocessing


from main import Model,load_model
from src.utils import footprint_extractor,get_import_mean_stressor,get_total_imports_region,recal_stressor_per_region,get_total_imports_region,multi_footprints_extractor,save_footprints,save_Coef_Row
from src.scenarios import DICT_SCENARIOS
from src.advance import extract_data
from src.figures import*
from src.stressors import GHG_STRESSOR_NAMES,GHG_AND_MATERIALS_PARAM,MATERIAL_STRESSOR_NAMES,STRESSOR_DICT_GHG_MAT_DETAIL

### Loading/building the model

In [5]:
#c'est ici que l'on choisis le niveau d'aggrégation, ils sont définis dans "data/aggragation/NOM_DE_L_AGGREG.xlsx"
model_no_cap = Model(aggregation_name="MatMat_sectors_all_countries",stressor_params=GHG_AND_MATERIALS_PARAM,exiobase_vanilla=True,calib=True)

#Le paramètre "exiobase_vanilla" permet d'ignorer la recalibration sur la capital que tu avais mise en place, histoire d'avoir de résultats plus facilement comparable
# si "exiobase_vanilla" = False alors la recalibration a lieu

#ignore performace warnings
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)

Loading data... (may take a few minutes)
Data loaded successfully !


### Building the counterfactual scenarios

In [ ]:
scenario= "INDC"   #the scenarios of IMACLIM we will be using
YEAR=2050

model_no_cap.new_counterfactual("IMACLIM_all_"+YEAR+"_"+scenario, DICT_SCENARIOS["emissivity_IMACLIM"], year=YEAR,scenario=scenario)
model_no_cap.modify_counterfactual("IMACLIM_all_"+YEAR+"_"+scenario,DICT_SCENARIOS["technical_change_IMACLIM"], year=YEAR,scenario=scenario)
# We have created a counterfactual scenario named "IMACLIM_all_2050_INDC" for the year 2050 based on IMACLIM trends

In [ ]:
# here we build the different variation of our trade sceanrios for both the base year and 2050

# for the 2050 trade scenario we use the method "modify counterfactual" to apply the scenarios to the already existing counterfactual "IMACLIM_all_2050_INDC"

model_no_cap.new_counterfactual("best_final",DICT_SCENARIOS["best final demand"])
model_no_cap.new_counterfactual("best",DICT_SCENARIOS["best"])
model_no_cap.modify_counterfactual("IMACLIM_all_2050_INDC",DICT_SCENARIOS["best final demand"],new_name="best_final_2050")
model_no_cap.modify_counterfactual("IMACLIM_all_2050_INDC",DICT_SCENARIOS["best"],new_name="best_2050")
model_no_cap.new_counterfactual("best_scope_1",DICT_SCENARIOS["best"],scope=1)
model_no_cap.modify_counterfactual("IMACLIM_all_2050_INDC",DICT_SCENARIOS["best"],new_name="best_2050_scope_1",scope=1)

In [ ]:
model_no_cap.save()

### Producing CoefRoW excels

In [6]:
Model_name="MatMat_sectors_all_countries_without_capital" # this is the name that will be used to name the excels
Path="../Outputs/CoefRow/{}.xlsx".format(Model_name) # path to save excels 

In [7]:
# opening the excels SRIO sheets to be able to translate from hybrid to monetaire units 
# make the necesssary adjustements to the path

SRIO_hyb=pd.read_excel("../Data/MatMat/SRIO_FR 2015.xlsx",sheet_name="hybride",header=[0,1,2],index_col=[0,1,2,3])
SRIO_monetaire=pd.read_excel("../Data/MatMat/SRIO_FR 2015.xlsx",sheet_name="monetaire",header=[0,1,2],index_col=[0,1,2,3])

In [8]:
# isolating the gross outputs
X_hyb=SRIO_hyb.loc[:,(slice(None),slice(None),"x")]
X_monetaire=SRIO_monetaire.loc[:,(slice(None),slice(None),"x")]

In [9]:
Transition_hybride_monetaire_RoW=(X_monetaire/X_hyb).fillna(0).loc["RoW"]
Transition_hybride_monetaire_RoW=Transition_hybride_monetaire_RoW.reset_index(["category","sub_category"])[("Supply","Supply","x")].fillna(0)

In [10]:
save_Coef_Row(model_no_cap,Transition_hybride_monetaire_RoW,Path,stressor_lists=STRESSOR_DICT_GHG_MAT_DETAIL)